# Instagram Filters App

## Load the necessary packages

In [1]:
# -*- coding: utf-8 -*-
import cv2
import numpy as np

# import tkinter and tkinter.filedialog
from tkinter import Tk     # from tkinter import Tk for Python 3.x
from tkinter.filedialog import askopenfilename


In [2]:
def dummy(value):
    pass

We keep the root window from appearing.

Show an "Open" dialog box and return the path to the selected file

In [3]:
def select_image():
    
    Tk().withdraw()
    
    filename = askopenfilename()
    return filename

## Define the convolution kernels

- Identity kernel: output is identical to the input. It takes numpy arrays

- Sharpen_kernel: It gives the most weight to the anchor points and gives less weight to the pixels around it.

- Blurring kernels:
    - the larger the numbers in the Gaussian Kernel function, the more intense the blur is going to be. 

    - The getGaussianKernel function returns a 1D Gaussian kernel, but for image filtering, I need a 2D kernel.
    
- I'll  to use cv2.getGaussianKernel(ksize, sigma) and then create a 2D kernel by multiplying two 1D kernels (one transposed)

In [4]:
identity_kernel = np.array([[0,0,0],[0,1,0],[0,0,0]])

sharpen_kernel = np.array([[0,-1,0], [-1,5,-1],[0, -1,0]])

In [5]:
gaussian_kernel1 = cv2.getGaussianKernel(3, 0)
gaussian_kernel1 = np.outer(gaussian_kernel1, gaussian_kernel1.T)

gaussian_kernel2 = cv2.getGaussianKernel(5, 0)
gaussian_kernel2 = np.outer(gaussian_kernel2, gaussian_kernel2.T)

- I introduce the "average filter" or box_kernel. 
    - It's called like that because it takes the average of the pixels inside the window. 
        - So, in a 3x3 matrix, I have 9 numbers total, then the filter give each number 1/9 of the weight. 
    - The first parameter in this function is the array.
    - The second is the array data type (in this case it's float 32).
    - Then we divide by 9 because there are 9 numbers. 

In [6]:
box_kernel = np.array([[1, 1, 1], [1, 1, 1], [1, 1, 1]], np.float32) / 9.0
# (1 + 2 + 3 + 4 + 5) / 5 = 1 * 1/5 + 2 * 1/5 + 3 * 1/5 + 4 * 1/5 + 5 * 1/5

In [7]:
# I create an array that stores all kernels
kernels = [identity_kernel, sharpen_kernel, gaussian_kernel1, gaussian_kernel2, box_kernel]

## Load and Read an image

In [8]:
# Load an image
image_path = select_image()
if image_path:
    color_original = cv2.imread(image_path)
    if color_original is None:
        print("Error loading image")
        exit()
    gray_original = cv2.cvtColor(color_original, cv2.COLOR_BGR2GRAY)
else:
    print("No file selected")
    exit()

## Create the UI (Window and Trackbars)

In [10]:
cv2.namedWindow('app')

# arguments: trackbarName, windowName, value (initial value), count (max value), on Change (event handler)
cv2.createTrackbar('contrast', 'app', 1, 100, dummy)

# name=brightness, initial value=50, max value=100, event handler=dummy
cv2.createTrackbar('brightness', 'app', 50, 100, dummy)
# Filter bar
cv2.createTrackbar('filter', 'app', 0, len(kernels)-1, dummy) # here I updated the filter bar so that it includes all the kernels I am going to create. 
# also, the -1 is because we index an array at 0, so 2 elements actually stop at 1. We need to count one less. 
# Grayscale bar
cv2.createTrackbar('grayscale', 'app', 0, 1, dummy)

## Main UI loop

- here we are creating a count for it to save the image as "output 1, output 2" anmd so on. 
- the function to apply a particular kernel is called "filter2D"
    - This function takes the original picture (here, 'color-original'), the other parameter is "depth". Here, we put -1 to just copy the depth of the color_original. 
    - The last parameter is the color filter that we want to apply. In tihs case, it is an index of kernels.

- Important: because the filters and the brightness and contrast build on each other,brightness and contrast are going to overwrite the filters that we apply.

    - So we need to chain them together, so that we apply the filters and then we apply the brightness and contrast on top of these. 


In [11]:
# main UI loop
count = 1

while True:
    # read all the trackbar values
    grayscale = cv2.getTrackbarPos('grayscale', 'app')
    contrast = cv2.getTrackbarPos('contrast', 'app')
    brightness = cv2.getTrackbarPos('brightness', 'app')
    kernel_idx = cv2.getTrackbarPos('filter', 'app')
    
    # Apply the kernel to color and grayscale images
    color_modified = cv2.filter2D(color_original, -1, kernels[kernel_idx])
    gray_modified = cv2.filter2D(gray_original, -1, kernels[kernel_idx])

    # Apply brightness and contrast
    color_modified = cv2.addWeighted(color_modified, contrast, np.zeros_like(color_original), 0, brightness - 50)
    gray_modified = cv2.addWeighted(gray_modified, contrast, np.zeros_like(gray_original), 0, brightness - 50)

    # Check grayscale toggle to decide which image to display and save
    if grayscale == 0:
        cv2.imshow('app', color_modified)
    else:
        cv2.imshow('app', gray_modified)

    # wait for keypress (100 milliseconds)
    key = cv2.waitKey(100)
    if key == ord('q'):
        break
    elif key == ord('s'):
        # Save image
        if grayscale == 0:
            cv2.imwrite('output-{}.jpg'.format(count), color_modified)
        else:
            cv2.imwrite('output-{}.jpg'.format(count), gray_modified)
        count += 1

# Window cleanup
cv2.destroyAllWindows()
